Enoncé et consignes
===================

Le propos de ce projet est de finaliser l'implantation d'un parser en dépendances par transitions à partir d'une implémentation limitée au tagging qui est proposée ci-dessous. 

Ce parser s'appuie sur un modèle d'analyse neuronal où la phrase d'entrée est encodée par un bi-lstm, ce qui servira de features pour prendre les décisions de transition.

Le parser utilisera l'algorithme arc-standard et utilisera une méthode de recherche de solutions gloutonne.

Le jeu de données utilisé est une verions du corpus sequoia : 
    https://gforge.inria.fr/frs/?view=shownotes&group_id=3597&release_id=9064.
qui a la propriété d'être projectif. 
Le fichier de données utilisé est `sequoia-corpus.np_conll`



Modèle d'apprentissage
---------------------
Le parser à réaliser sera un hybride entre celui proposé par Kipperwasser et Golbderg (2016)
et le système multitâche pour l'analyse en constituants décrit par Coavoux et Crabbé (2017).




Consignes spécifiques:
---------------------

1. Lire attentivement le code proposé ci-dessous et en discuter entre vous. Vous pouvez exécuter et tester le code existant il s'agit d'un tagger parfaitement fonctionnel.
2. Changer le système de transition :
    * Modifier la méthode `index_symbols` pour qu'elle encode sur le modèle des actions shift les actions de type left arc right arc sous forme de couple à deux éléments associés à des entiers
    * Implémenter les méthodes `left_arc`, `right_arc` 
    * Modifier la méthode `oracle` pour qu'elle implémente l'oracle statique arc-standard 
    * Modifier la méthode `forward` pour que le critère d'arrêt de la boucle corresponde au critère d'arrêt du système de transition que vous utilisez et que les transitions left-arc et right-arc soient effectivement utilisées. 
3. Implémenter une méthode `predict_tree(...)` inspirée de `predict_tags` qui renvoie l'arbre de dépendances prédit
sous forme d'un objet `DependencyTree`.
4. Implémenter une fonction `eval_parser` inspirée de `eval_tagger` qui renvoie le LAS et l'UAS de votre parser.    
5. Features pour le parser. A ce stade, le parser devrait renvoyer des résultats la plupart du temps incorrects car la décision d'analyse est prise uniquement sur base de la représentation du premier élément du buffer.
   * Modifier le modèle pour que cette décision s'appuie également sur les représentations des deux (ou trois) éléments situés sur la pile.
   * Evaluer l'impact de vos modifications et comparer par rapport à ce qui précède. Reporter vos UAS/LAS à chaque fois  
6. Gestion des mots inconnus. Suite à vos premières évaluations, vous aurez peut-être remarqué que le tagger ou le parser a des performances relativement moyennes. Pour améliorer la situation, on propose d'incorporer un module de gestion des mots inconnus.
    * Implanter une méthode `word_dropout(...,alpha=1.0)` qui prend en paramètre une liste de mots et qui renvoie cette liste avec certains mots remplacés par le token du mot inconnu. En général on choisit de remplacer un mot de basse fréquence $w$ par le token inconnu avec la probabilité $p = \frac{\alpha}{count(w)} $ où $\alpha \in [0,1]$ mais vous êtes libre de choisir votre méthode.  
    * Implanter une classe de modèle de caractère équipée d'une méthode `forward(...)` qui prend en entrée une string et qui renvoie un vecteur calculé par un modèle RNN de caractères. Intégrer ce module au modèle d'analyse syntaxique pour améliorer la gestion des mots inconnus (**Exercice plus compliqué**, mais vous pouvez vous inspirer de tutoriels existants sur le web)
    * Evaluer l'impact de vos modifications et comparer par rapport à la baseline. Reporter vos UAS/LAS à chaque fois    



In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import sys
import numpy as np
from collections import Counter
from random import random

from tqdm import tqdm #optional, for progress bars in notebook.

Structure de données annexe
====================

In [2]:
class DependencyTree:

    DUMMY_ROOT = '$ROOT$'
    
    def __init__(self,wordlist,edges=None):
        
            self.wordlist = wordlist                  #hyp: DUMMY ROOT is wordlist[0]  
            self.edges    = edges if edges else []    #edges are 4-tuples (govidx,label,deptag,depidx)
            self.edges.sort(key = lambda x:x[3])
                
    def tag_list(self):
        return [deptag for (gidx,deplabel,deptag,didx) in self.edges]
    def word_list(self):
        return self.wordlist
    def dlabel_list(self):
         return [deplabel for (gidx,deplabel,deptag,didx) in self.edges]
    
    #aux functions useful for oracle
    def get_edge(self,gov_idx,dep_idx):
       #gets the edge (with labels) from gov_idx to dep_idx 
       #returns either a singleton or an empty list
       return [ (gidx,x,y,didx) for (gidx,x,y,didx) in self.edges \
                                if gidx == gov_idx and didx == dep_idx]

    def dom_edges(self,gov_idx):
         #gets all edges couples from gov_idx to some dependant node
         return [(gidx,didx) for (gidx,_,_,didx) in self.edges if gidx == gov_idx]
        
    @staticmethod
    def read_tree(istream):
        """
        Reads a Conll-u tree and returns it as a DepTree object
        returns None if no tree has been read
        """
        words = []
        edges = []
        for line in istream:
            line = line.split("#")[0]
            if line and not line.isspace():
                values = line.split()
                words.append(values[1])
                edges.append( (int(values[6]),values[7], values[3], int(values[0])) )
            elif words:
                return DependencyTree([DependencyTree.DUMMY_ROOT] + words,edges)
        return DependencyTree([DependencyTree.DUMMY_ROOT] + words,edges) if words else None
    
    
    def __str__(self):
        """
        Pretty prints a tree for debug
        Returns: a string
        """
        lines =  [(didx,self.wordlist[didx],dtag,dlabel,gidx) for (gidx,dlabel,dtag,didx) in self.edges] 
        lines.sort(key=lambda x:x[0])
        return '\n'.join( [ "%d\t%s\t%s\t%s\t%d"%(L) for L in lines ] )

Lecture des données
====================

In [3]:
from random import shuffle

def is_projective(root_idx,edges):
    rspan = [root_idx]
    children = [ didx for (gidx,_,_,didx) in edges if gidx==root_idx ]
    for child in children:
        proj,span = is_projective(child,edges)
        if not proj:
            return False,[]
        rspan.extend(span)
    rspan.sort()
    return  ( all([ jdx == idx+1 for idx,jdx in zip(rspan,rspan[1:])]),rspan )
    
def read_trees(filename):

    istream = open(filename)
    
    ilist = []
    dtree = DependencyTree.read_tree(istream)
    while dtree is not None:
        proj,span = is_projective(0,dtree.edges)
        if proj:
            ilist.append(dtree)
        #else:
        #    print('[warning] non projective tree skipped')
        dtree = DependencyTree.read_tree(istream)
        
    istream.close()
    return ilist
        
train_treebank = read_trees("/Users/bcrabbe/parsing-at-diderot/data/train.French.gold.conll")
valid_treebank = read_trees("/Users/bcrabbe/parsing-at-diderot/data/dev.French.gold.conll")
test_treebank  = read_trees("/Users/bcrabbe/parsing-at-diderot/data/test.French.gold.conll")

print('#training trees',len(train_treebank))
print('#validation trees',len(valid_treebank))
print('#test trees',len(test_treebank))


#training trees 14754
#validation trees 1235
#test trees 2540


Codage
======

In [4]:
def make_sym2idx(wordlist,unk_symbol=None):
    """
    Creates a dictionary mapping symbols to integers
    Args:
       wordlist  (list): a list of strings
       unk_symbol (str): a special default string for symbols unknown to this dictionary
    Returns:
        a dict string => idx mapping words to integer indexes
    """
    if unk_symbol:
        wordlist.append(unk_symbol)
    wordset = set(wordlist)
    return dict(zip(wordset,range(len(wordset)))) 

def code_sequence(symlist,sym2idx,unk_symbol=None):
    """
    Maps a list of string to a list of int (encodes the sentence on integers) 
    Args:
        symlist  (list): a list of strings
        sym2idx  (dict): a dictionary mapping strings to int
        unk_symbol(str): a special default string for the dictionary sym2idx
    Returns a list of int as torch.tensor where words are mapped to their integer indexes.
    """
    def normal_form(symbol):
        return symbol if symbol in sym2idx else unk_symbol
    
    code_list = [sym2idx[normal_form(symbol)] for symbol in symlist]
    return torch.tensor(code_list, dtype=torch.long)

Modèle
=======

In [5]:
class ArcStandardParser(nn.Module):
      
    #ACTIONS
    SHIFT = "S"
    LARC  = "L"
    RARC  = "R"
    
    #DUMMY TAG
    DTAG  = 'ROOT'
    
    def __init__(self,embedding_dim,lstm_memory_dim,config_embedding_dim):
        """
        Args:
            embedding_dim        (int): size of word embeddings
            lstm_memory_dim      (int): size of the sentence encoder lstm memory 
            config_embedding_dim (int): size of the hidden layer of the output MLP.
        """
        super(ArcStandardParser, self).__init__()
        self.embedding_dim        = embedding_dim
        self.lstm_memory_dim      = lstm_memory_dim
        self.config_embedding_dim = config_embedding_dim

        self.char_embedding_dim   = 32
        self.char_lstm_memory_dim = 64 
        
    ### INDEXING ###
    def index_symbols(self,treebank,unk_word,unk_char):
        """
        Indexes the x and y symbols on integers
        Args:
           treebank   (list): a list of DependencyTree objects
           unk_word (string): a symbol to use for unk words
           unk_char (string): a symbol to use for unk char
        Returns:
           (a dict word -> int, a dict action -> int)
        """
        wlist      = []   
        taglist    = [ArcStandardParser.DTAG]
        actionlist = [(ArcStandardParser.SHIFT,None)]
        charset    = set([])
        for dtree in treebank:
            for w in dtree.word_list():
                charset.update(list(w))
            wlist.extend(dtree.word_list())
            taglist.extend(dtree.tag_list())
            actionlist.extend([ (ArcStandardParser.LARC,dlab) for dlab in dtree.dlabel_list() ] )
            actionlist.extend([ (ArcStandardParser.RARC,dlab) for dlab in dtree.dlabel_list() ] )
        w2idx         = make_sym2idx(wlist,unk_symbol=unk_word)
        tag2idx       = make_sym2idx(taglist)
        wcounts       = Counter(wlist)
        action2idx    = make_sym2idx(actionlist)
        char2idx      = make_sym2idx(wlist,unk_symbol=unk_char)
        return w2idx,tag2idx,action2idx,wcounts,char2idx
    
    ### MODEL STRUCTURE AND INFERENCES ###
    def allocate_structure(self,nWords,nTags,nActions,nChars):
        """
        Allocates memory for the parser network params.
        Args:
           nWords  (int):number of words in the sym2idx dict
           nTags   (int):number of tags  in the parser
           nActions(int):number of actions in the parser
           nChars  (int):number of chars inthe char2idx dict
        """
        self.word_embeds           = nn.Embedding(nWords, self.embedding_dim)
        self.tagger_bilstm         = nn.LSTM(self.embedding_dim+self.char_lstm_memory_dim, self.lstm_memory_dim,num_layers=1,bidirectional=True)
        self.parser_bilstm         = nn.LSTM(self.lstm_memory_dim*2, self.lstm_memory_dim,num_layers=1,bidirectional=True)
        self.tagger_lstm2hidden    = nn.Linear(self.lstm_memory_dim*2,self.config_embedding_dim)
        self.hidden2tags           = nn.Linear(self.config_embedding_dim,nTags)
        self.parser_lstm2hidden    = nn.Linear(self.lstm_memory_dim*8,self.config_embedding_dim)
        self.hidden2actions        = nn.Linear(self.config_embedding_dim,nActions)
       
        self.char_embeds    = nn.Embedding(nChars, self.char_embedding_dim)
        self.char_bilstm    = nn.LSTM(self.char_embedding_dim, self.char_lstm_memory_dim,num_layers=1,bidirectional=True)        
        self.char_lstm2word = nn.Linear(self.char_lstm_memory_dim*2,self.char_lstm_memory_dim)
    
    
    def word_dropout(self,toklist,alpha=1.0):
        """
        Replaces each word in toklist with probability Lambda / counts(w)
        Args:
            toklist (list): a list of strings
            alpha  (float): a real in [0,1]. a word w is replaced with prob = alpha/counts(w)
        Returns:
            a list of strings with some strings replace by the unk token
        """
        def sample_word(wordform,unk_form):
            r = random()
            if r < alpha/self.wcounts[wordform]:
                return unk_form
            return wordform
    
        if alpha == 0:     #no dropout
            return toklist
        
        return [sample_word(token,self.unk_word) for token in toklist]
         
    def train_model(self,train_set,validation_set,epochs=10,learning_rate=0.1,unk_word='##UNK##',unk_char='@@',alpha=0.5):
                    
        self.w2idx, self.tag2idx,self.a2idx,self.wcounts,self.char2idx = self.index_symbols(train_set,unk_word,unk_char)
        
        self.reva2idx   = [action for action,idx in sorted(self.a2idx.items(),key = lambda x : x[1])]
        self.revtag2idx = [tag for tag,idx in sorted(self.tag2idx.items(),key = lambda x : x[1])]
        
        self.unk_word,self.unk_char = unk_word,unk_char
        self.allocate_structure(len(self.w2idx),len(self.tag2idx),len(self.a2idx),len(self.char2idx))
        
        loss_function = nn.NLLLoss()
        optimizer     = optim.Adam(self.parameters(), lr=learning_rate)
        
        min_loss      = np.iinfo(np.int32).max
        for epoch in range(epochs):
            
            NLL = 0.0
            for deptree in tqdm(train_set): 
                
                self.zero_grad()                     
                
                X               = deptree.word_list()
                xtagger,xparser = self.forward_encoding(X,alpha)

                ypred,yref      = self.forward_tagger(xtagger,oracle_tree=deptree) # ypred is a sequence of y-softmaxed-probs for each step in the inference                      
                loss            = loss_function(ypred, yref)
                NLL            += loss.item()
                loss.backward(retain_graph=True)
                    
                ypred,yref      = self.forward_parser(xparser,oracle_tree=deptree)
                loss            = loss_function(ypred, yref)
                NLL            += loss.item()
                loss.backward()
              
                optimizer.step()
                
            print("\n[train]      Epoch %d, NLL = %f"%(epoch,NLL/len(train_set)),file=sys.stderr)
    
            with torch.no_grad():
                NLL = 0.0
                for deptree in validation_set:
                     X               = deptree.word_list()
                     xtagger,xparser = self.forward_encoding(X)
                        
                     ypred,yref      = self.forward_tagger(xtagger,oracle_tree=deptree)      # ypred is a sequence of y-softmaxed-probs for each step in the inference
                     loss            = loss_function(ypred, yref) 
                     NLL            += loss.item()
                    
                     ypred,yref      = self.forward_parser(xparser,oracle_tree=deptree)
                     loss            = loss_function(ypred, yref)
                     NLL            += loss.item()
                if NLL < min_loss:
                    torch.save(self.state_dict(), 'parsing_model.wt')        
                print("[validation] Epoch %d, NLL = %f\n"%(epoch,NLL/len(validation_set)),file=sys.stderr)  
            
        self.load_state_dict(torch.load('parsing_model.wt'))

        
    def forward_encoding(self,wordlist,alpha=0.0):
        """
        Performs the forward pass for encoding the input only.
        Args:
           wordlist   (list): a list of strings
           alpha     (float): 0 <= alpha <= 1 float for word dropout
        Returns:
           two lists of vectors encoding the sentence to be used for tagging and parsing the input
           as torch.tensor
        """        
        xwords                = code_sequence(self.word_dropout(wordlist,alpha),self.w2idx,unk_symbol=self.unk_word) 
        xword_embedded        = self.word_embeds(xwords)
        xchar_embedded        = []
        for w in wordlist:
            char_codes        = code_sequence(list(w),self.char2idx,unk_symbol=self.unk_char) 
            char_embedded     = self.char_embeds(char_codes)
            _,(char_hidden,_) = self.char_bilstm(char_embedded.view(len(w), 1, -1), None)
            xchar_embedded.append(self.char_lstm2word(char_hidden.view(2*self.char_lstm_memory_dim)))
        
        xchar_embedded        = torch.stack(xchar_embedded)
        xcodes                = torch.cat([xword_embedded,xchar_embedded],1)

        xtagger, hidden_tag   = self.tagger_bilstm(xcodes.view(len(xcodes), 1, -1), None)
        xparser, hidden_par   = self.parser_bilstm(xtagger, None)
        return (xtagger,xparser)
        
        
    def forward_tagger(self,xtagger,oracle_tree=None):
        """
        Performs the forward pass for tagging only.
        Args:
           xtagger               (list): a list of vectors,one per word,encoding the input sentence
           oracle_tree (DependencyTree): an optional Dependency tree used as oracle
        Returns:
           a list of tags (one per word, including the dummy word)
           or
           a list of softmax distributions as torch.tensor and the list of reference tags if an oracle_tree is provided
        """
        
        hidden        = F.relu(self.tagger_lstm2hidden(xtagger.squeeze()))
        softmaxin     = self.hidden2tags(hidden)
        log_softmaxes = F.log_softmax(softmaxin, dim=1)  

        if oracle_tree:
            ref_tags = [ArcStandardParser.DTAG] + oracle_tree.tag_list()
            ref_tags = code_sequence(ref_tags,self.tag2idx)
            return (log_softmaxes,ref_tags)
        else:
            max_probs,  argmaxlist  = torch.max(log_softmaxes,1)
            pred_tags               = [self.revtag2idx[argmax] for argmax in argmaxlist]
            return pred_tags
        
    def forward_parser(self,xparser,oracle_tree=None):
        """
        Performs the forward pass for parsing only.
        Args:
           xparser               (list): a list of vectors,one per word,encoding the input sentence
           oracle_tree (DependencyTree): an optional Dependency tree used to drive the oracle
        Returns:
           the final configuration
           or
           a list of softmax distributions, and the list of reference actions if an oracle_tree is provided
        """        
        
        yprobs     = [] #sequence of predictions
        yref       = [] #sequence of gold actions
        
        config     = self.init_configuration(len(xparser))
        S,B,A      = config
        
        while B or len(S) >= 2:
            
            #predict action probs with MLP
            S1 = xparser[ S[-1] ] if S          else torch.zeros(1,self.lstm_memory_dim*2)
            S2 = xparser[ S[-2] ] if len(S) > 1 else torch.zeros(1,self.lstm_memory_dim*2)
            S3 = xparser[ S[-3] ] if len(S) > 2 else torch.zeros(1,self.lstm_memory_dim*2)
            B0 = xparser[ B[0]  ] if B          else torch.zeros(1,self.lstm_memory_dim*2)
        
            xfeatures     = torch.cat([S3,S2,S1,B0],1)
            hidden        = F.relu(self.parser_lstm2hidden(xfeatures))
            
            softmaxin     = self.hidden2actions(hidden) + self.run_time_mask(config)
            log_softmaxes = F.log_softmax(softmaxin, dim=1)  
            
            #best action
            if oracle_tree:
                selected_action      = self.oracle(config,oracle_tree)
                yref.append(selected_action)
                yprobs.append(log_softmaxes)
            else:
                max_prob,  argmax    = torch.max(log_softmaxes,1)
                selected_action      = self.reva2idx[argmax]

            #update config
            act_type,act_label   = selected_action
            if act_type   == ArcStandardParser.SHIFT:
                config           = self.shift(config)
            elif act_type == ArcStandardParser.LARC:
                config           = self.left_arc(config,act_label)
            elif act_type == ArcStandardParser.RARC:
                config           = self.right_arc(config,act_label)
            S,B,A = config
                    
        if oracle_tree:
            yprobs = torch.cat(yprobs)
            yref   = code_sequence(yref,self.a2idx)
            return (yprobs,yref)
        else:
            return config
        
    def predict_tree(self,wordlist):
        """
        Actually performs the tagging and parsing of a sentence.
        Args:
            wordlist (list): a list of strings
        Returns:
            a tree  (DependencyTree): the dep tree and its tags
        """
        with torch.no_grad():
            xtagger,xparser  = self.forward_encoding(wordlist)
            taglist          = self.forward_tagger(xtagger)
            S,B,A            = self.forward_parser(xparser) 
            edges            = [(gidx,deplabel,taglist[didx],didx)  for (gidx,deplabel,didx) in A]
            return           DependencyTree(wordlist,edges)    
        
    ### TRANSITION SYSTEM ###
    def init_configuration(self,N):
        """
        Creates an init configuration for an arc standard parser.
        A configuration is a triple (stack_idxes,buffer_idxes,dep_arcs)
        Args:
            N  (int): the length of the input
        Returns:
            the init configuration
        """
        return ([0],list(range(1,N)),[]) 
        # configs stack and buffer are filled with integer indexes of word positions
        # stack starts with [0] the dummy root node
        
    def shift(self,configuration):
        """
        Performs shift on a configuration and returns the result
        Args:
           configuration (tuple): a configuration to be shifted
        Returns: 
           the shifted configuration
        """
        stack,buffer,arcs = configuration
        return (stack + [buffer[0]],buffer[1:],arcs)
                
    def left_arc(self,configuration,deplabel):
        """
        Performs a left arc action on a configuration and returns the result
        Args:
           configuration (tuple): a configuration
           deplabel     (string): a string labelling the arc
        Returns: 
           the resulting configuration
        """
        stack,buffer,arcs = configuration
        i = stack[-2]
        j = stack[-1]
        return (stack[:-2]+[j],buffer,arcs+[(j,deplabel,i)])
        
    def right_arc(self,configuration,deplabel):
        """
        Performs a right arc action on a configuration and returns the result
        Args:
           configuration (tuple): a configuration
           deplabel     (string): a string labelling the arc
        Returns: 
           the resulting configuration
        """
        stack,buffer,arcs = configuration
        i = stack[-2]
        j = stack[-1]
        return (stack[:-1],buffer,arcs+[(i,deplabel,j)])

    def run_time_mask(self,configuration,log_mask=True):
        """
        The run time mask returns a mask where impossible actions have 0 score.
        The mask is a vector of mask pseudo scores with dimensions comparable to the output action score vector.
        Impossible actions are filled with a -inf value, and possible actions with a 0 value.
        
        The mask can be added to action scores prior to softmax.
        Using the mask ensures that we have no fatal error while performing oracle-less runtime search.
        Args:
           configuration (tuple): a configuration
        Kwargs:
            log_mask      (bool): bool stating if the mask is in logit_space or in probabilistic space.
        Returns:
            torch.tensor of size num Actions filled with masking values
        """
        m0,m1 = (-float('inf'), 0.0) if log_mask else (0.0,1.0)
        
        def valmask(atype,stack_size,buffer_size,configuration):
            if atype in [ArcStandardParser.LARC,ArcStandardParser.RARC] and stack_size < 2:
                return m0
            elif atype == ArcStandardParser.LARC and stack_size >= 2:
                S,B,A = configuration
                if S[-2] == 0:
                    return m0
            elif atype == ArcStandardParser.SHIFT and buffer_size == 0:
                return m0
            return m1
            
        S,B,A       = configuration
        stack_size  = len(S)
        buffer_size = len(B)
        return torch.tensor([ valmask(atype,stack_size,buffer_size,configuration)   for (atype,alabel) in self.reva2idx])  

    def oracle(self,config,deptree):
        """
        Deterministic function translating a dependency tree into a derivation sequence step by step.
        Arc standard static oracle.
        Args:
            config           (tuple): a configuration
            deptree (DependencyTree): the tree used as ground truth
        Returns:
            the action as a tuple (ActionType,ActionLabel)
        """
        def node_is_complete(nidx,ref_tree,dep_arcs):
            #says if all ref arcs leaving from this node in the reference are already in dep arcs (true) or not (false)
            ref_edges  = set(ref_tree.dom_edges(nidx))
            pred_edges = set( [(gidx,didx) for (gidx,_,didx) in dep_arcs if gidx == nidx] ) 
            return ref_edges.issubset(pred_edges) 
        
        S,B,dep_arcs = config
        if len(S) >= 2:
            e = deptree.get_edge(S[-2],S[-1])
            if e and node_is_complete(S[-1],deptree,dep_arcs):
                gidx,deplabel,deptag,didx = e[0]
                return (ArcStandardParser.RARC,deplabel)
            
            e = deptree.get_edge(S[-1],S[-2]) 
            if S[-2] != 0 and e and node_is_complete(S[-2],deptree,dep_arcs):
                gidx,deplabel,deptag,didx = e[0]
                return (ArcStandardParser.LARC,deplabel)
        if B: 
            return (ArcStandardParser.SHIFT,None)
    
        return None #terminate
    

Evaluation
==========

In [6]:
def eval_parser(model,test_treebank,display=False):
    
    las_correct = 0
    uas_correct = 0
    tag_correct = 0
    
    for dtree in test_treebank:
        
        X           = dtree.word_list()
        ptree       = model.predict_tree(X)
        
        ref_edges   = set( [(gidx,deplabel,didx) for (gidx,deplabel,_,didx) in dtree.edges ] )
        pred_edges  = set( [(gidx,deplabel,didx) for (gidx,deplabel,_,didx) in ptree.edges ] )
        las_correct+= len(ref_edges.intersection(pred_edges))/len(ref_edges)
        
        ref_edges   = set( [(gidx,didx) for (gidx,deplabel,_,didx) in dtree.edges ] )
        pred_edges  = set( [(gidx,didx) for (gidx,deplabel,_,didx) in ptree.edges ] )
        uas_correct+= len(ref_edges.intersection(pred_edges))/len(ref_edges)
        
        ref_tags     = dtree.tag_list()
        pred_tags    = ptree.tag_list()
        tag_correct += sum([p==r for p,r in zip(pred_tags,ref_tags)])/len(ref_edges)
        
        if display:
            print(ptree,'\n')
    
    print('Model accurracy LAS: %f ; UAS: %f ; Tag accuracy: %f'%(las_correct/len(test_treebank),uas_correct/len(test_treebank),tag_correct/len(test_treebank)))
    


In [7]:
m = ArcStandardParser(32,64,100)
m.train_model(valid_treebank[:20],valid_treebank[:20],epochs=45,learning_rate=0.001,alpha=0.5)

100%|██████████| 20/20 [00:04<00:00,  5.52it/s]

[train]      Epoch 0, NLL = 5.219214
[validation] Epoch 0, NLL = 4.255364

100%|██████████| 20/20 [00:03<00:00,  5.39it/s]

[train]      Epoch 1, NLL = 4.136380
[validation] Epoch 1, NLL = 3.948225

100%|██████████| 20/20 [00:03<00:00,  5.45it/s]

[train]      Epoch 2, NLL = 3.815045
[validation] Epoch 2, NLL = 3.579101

100%|██████████| 20/20 [00:04<00:00,  5.14it/s]

[train]      Epoch 3, NLL = 3.411591
[validation] Epoch 3, NLL = 3.147165

100%|██████████| 20/20 [00:04<00:00,  4.99it/s]

[train]      Epoch 4, NLL = 3.087611
[validation] Epoch 4, NLL = 2.798483

100%|██████████| 20/20 [00:04<00:00,  4.89it/s]

[train]      Epoch 5, NLL = 2.740148
[validation] Epoch 5, NLL = 2.531080

100%|██████████| 20/20 [00:04<00:00,  4.99it/s]

[train]      Epoch 6, NLL = 2.489665
[validation] Epoch 6, NLL = 2.301267

100%|██████████| 20/20 [00:04<00:00,  4.85it/s]

[train]      Epoch 7, NLL = 2.273573
[validation] Epoch 7, NLL = 2.090828

100%|███

In [8]:
eval_parser(m,valid_treebank[:20],display=True)
#eval_parser(m,test_treebank,display=False)

1	Nous	CL	suj	2
2	prions	V	root	0
3	les	D	det	4
4	cinéastes	N	obj	2
5	et	C	coord	4
6	tous	A	mod	8
7	nos	D	det	8
8	lecteurs	N	dep.coord	5
9	de	P	p_obj	2
10	bien	ADV	mod	11
11	vouloir	V	obj.p	9
12	nous	CL	de_obj	14
13	en	CL	obj	14
14	excuser	V	obj	11
15	.	PONCT	ponct	2 

1	La	D	det	2
2	diffusion	A	suj	8
3	des	P+D	dep	2
4	prévisions	N	obj.p	3
5	météorologiques	A	dep_cpd	4
6	était	V	aff	8
7	fortement	ADV	mod	8
8	perturbée	V	root	0
9	,	PONCT	ponct	8
10	mardi	N	mod	8
11	7	A	mod	10
12	janvier	N	mod	10
13	,	PONCT	ponct	8
14	par	P	mod	8
15	le	D	det	16
16	mouvement	N	obj.p	14
17	de	P	dep	16
18	grève	N	obj.p	17
19	nationale	A	mod	18
20	de	P	dep	16
21	trente	D	det	24
22	-	PONCT	dep_cpd	21
23	six	D	dep_cpd	21
24	heures	N	obj.p	20
25	déclenché	V	dep	16
26	la	D	det	29
27	veille	N	dep_cpd	26
28	au	P+D	dep_cpd	26
29	soir	N	mod	25
30	,	PONCT	ponct	29
31	à	P	dep	29
32	l'	D	dep_cpd	31
33	appel	N	dep_cpd	31
34	des	P+D	dep	29
35	délégués	N	obj.p	34
36	CGT	N	mod	35
37	,	PONCT	coord	36
38	CFDT	N	dep.coord	37


In [9]:
#eval_parser(m,train_treebank,display=False)